# Preprocessing

## Step 1: Check missing images

In [1]:
import os.path
from datetime import date
import datetime


In [2]:
base_path = os.path.join("D:\\","Graduation_Thesis_Dataset")
S1_path = os.path.join(base_path, "S1_dataset")
L8_path = os.path.join(base_path, "L8_dataset")


In [3]:
years = [2017, 2018, 2019, 2020]
site_range = [0, 7, 8, 9,]
satellites = ['Landsat_8', 'S1']
Missing_image_paths = []
valid_image_paths = []
cdl_paths = []
for satellite in satellites:
  cur_path = None
  if satellite == 'Landsat_8':
    cur_path = L8_path
  elif satellite == 'S1':
    cur_path = S1_path
  for site in site_range:
    # print(cur_path)
    site_path = os.path.join(cur_path,  str(site))
    # print(cur_path)
    for year in years:
      year_path = os.path.join(site_path,str(year))
      cdlpath = "CDL_" + str(site) + '_' + str(year) + '.tif'
      cdlpath = os.path.join(year_path, cdlpath)
      if os.path.isfile(cdlpath):
        cdl_paths.append(cdlpath)
      start_date = date(year, 9, 10)
      for advancement in range(0, 12):
        cur_time = start_date + datetime.timedelta(days=advancement*24)
        cur_time_str = cur_time.strftime("%Y%m%d")
        image_path = os.path.join(year_path, satellite + '_' + str(site) + '_'+ cur_time_str + '.tif')
        if os.path.isfile(image_path):
          valid_image_paths.append(image_path)
        else:
          Missing_image_paths.append(image_path)
    



In [4]:
print('Valid images: ', len(valid_image_paths))

Valid images:  384


In [6]:
print('Missing images: ', len(Missing_image_paths))
print(Missing_image_paths)


Missing images:  0
[]


## Step 2 Fill in Nan Values

In [7]:
import cv2 as cv
from osgeo import gdal
from tqdm import tqdm
from sklearn.impute import KNNImputer
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [8]:
def replace_nan(image, threshold = 6, window = 5):
  x = image.shape[0]
  y = image.shape[1]
  for i in range (2, x - 2):
    for j in range(2, y - 2):
      if not np.isnan(image[i, j]):
        continue
      img_window = image[i-2:i+3, j-2:j+3]
      if np.count_nonzero(np.isnan(img_window)) > threshold:
        continue
      mean = np.nanmean(img_window)
      std = np.nanstd(img_window)
      image[i, j] = np.random.normal(mean, std)
  return image

In [9]:
NAN_image_path = []
for path in tqdm(valid_image_paths):
  new_img = gdal.Open(path)
  band_cnt = new_img.RasterCount
  channel_list = []
  is_nan_too_much = False
  for b in range(1, band_cnt + 1):
    img_array = np.array(new_img.GetRasterBand(b).ReadAsArray())
    nan_percent = np.count_nonzero(np.isnan(img_array)) / (img_array.shape[0]*img_array.shape[1])
    # print('before: ',nan_percent)
    if nan_percent > 0.3:
      is_nan_too_much = True
      NAN_image_path.append(path)
    img_array = replace_nan(img_array)
    nan_percent = np.count_nonzero(np.isnan(img_array)) / (img_array.shape[0]*img_array.shape[1])
    # print('after: ',nan_percent)
    channel_list.append(img_array)
  output = np.moveaxis(np.stack(channel_list, axis=0), 0, -1)
  output_path = path.replace('.tif', '.npy')
  # print(output_path)
  # print(output.shape)
  np.save(output_path, output)




100%|██████████| 384/384 [1:57:40<00:00, 18.39s/it]  


In [10]:
Nan_paths = set(NAN_image_path)
print(Nan_paths)

{'D:\\Graduation_Thesis_Dataset\\L8_dataset\\7\\2019\\Landsat_8_7_20200108.tif', 'D:\\Graduation_Thesis_Dataset\\S1_dataset\\8\\2017\\S1_8_20171004.tif', 'D:\\Graduation_Thesis_Dataset\\L8_dataset\\0\\2020\\Landsat_8_0_20210508.tif', 'D:\\Graduation_Thesis_Dataset\\L8_dataset\\7\\2020\\Landsat_8_7_20210108.tif', 'D:\\Graduation_Thesis_Dataset\\S1_dataset\\8\\2017\\S1_8_20170910.tif', 'D:\\Graduation_Thesis_Dataset\\L8_dataset\\7\\2018\\Landsat_8_7_20180910.tif', 'D:\\Graduation_Thesis_Dataset\\L8_dataset\\9\\2020\\Landsat_8_9_20210508.tif', 'D:\\Graduation_Thesis_Dataset\\L8_dataset\\0\\2019\\Landsat_8_0_20200225.tif', 'D:\\Graduation_Thesis_Dataset\\L8_dataset\\7\\2018\\Landsat_8_7_20181215.tif', 'D:\\Graduation_Thesis_Dataset\\L8_dataset\\0\\2018\\Landsat_8_0_20180910.tif'}


### Performing Linear Interpolation

In [11]:
for satellite in satellites:
  cur_path = None
  if satellite == 'Landsat_8':
    cur_path = L8_path
  elif satellite == 'S1':
    cur_path = S1_path
  for site in site_range:
    # print(cur_path)
    site_path = os.path.join(cur_path,  str(site))
    # print(cur_path)
    for year in years:
      year_path = os.path.join(site_path,str(year))
      cdlpath = "CDL_" + str(site) + '_' + str(year) + '.tif'
      cdlpath = os.path.join(year_path, cdlpath)
      if os.path.isfile(cdlpath):
        cdl_paths.append(cdlpath)
      start_date = date(year, 9, 10)
      for advancement in range(1, 11):
        before_time = start_date + datetime.timedelta(days=(advancement - 1) * 24)
        before_time_str = before_time.strftime("%Y%m%d")
        cur_time = start_date + datetime.timedelta(days=advancement*24)
        cur_time_str = cur_time.strftime("%Y%m%d")
        after_time = start_date + datetime.timedelta(days=(advancement + 1) * 24)
        after_time_str = after_time.strftime("%Y%m%d")
        before_time_path = os.path.join(year_path, satellite + '_' + str(site) + '_'+ before_time_str + '.npy')
        cur_image_path = os.path.join(year_path, satellite + '_' + str(site) + '_'+ cur_time_str + '.npy')
        after_time_path = os.path.join(year_path, satellite + '_' + str(site) + '_'+ after_time_str + '.npy')
        cur_img = np.load(cur_image_path)
        nan_cnt_bef = np.count_nonzero(np.isnan(cur_img))
        before_image = np.load(before_time_path)
        after_image = np.load(after_time_path)
        new_array = np.array([before_image, after_image])
        cur_img = np.where(np.isnan(cur_img), np.nanmean(new_array, axis=0), cur_img)
        nan_cnt_after = np.count_nonzero(np.isnan(cur_img))
        assert(nan_cnt_after - nan_cnt_bef <= 0)
        print(satellite + '_' + str(site) + '_'+ after_time_str + '.npy: ', nan_cnt_bef-nan_cnt_after, ", ", (nan_cnt_bef - nan_cnt_after)/(cur_img.shape[0]*cur_img.shape[1]*cur_img.shape[2]))
        np.save(cur_image_path, cur_img)
        
        

C:\Users\11027\AppData\Local\Temp/ipykernel_31868/3244010852.py:33: RuntimeWarning: Mean of empty slice
  cur_img = np.where(np.isnan(cur_img), np.nanmean(new_array, axis=0), cur_img)


Landsat_8_0_20171028.npy:  642 ,  7.227580989435573e-05
Landsat_8_0_20171121.npy:  481520 ,  0.05420910900362955
Landsat_8_0_20171215.npy:  36 ,  4.052849152954527e-06
Landsat_8_0_20180108.npy:  95796 ,  0.010784631596011996
Landsat_8_0_20180201.npy:  90 ,  1.0132122882386318e-05
Landsat_8_0_20180225.npy:  0 ,  0.0
Landsat_8_0_20180321.npy:  108 ,  1.2158547458863581e-05
Landsat_8_0_20180414.npy:  0 ,  0.0
Landsat_8_0_20180508.npy:  194448 ,  0.02189078922482505
Landsat_8_0_20180601.npy:  0 ,  0.0
Landsat_8_0_20181028.npy:  0 ,  0.0
Landsat_8_0_20181121.npy:  434334 ,  0.04889694955553754
Landsat_8_0_20181215.npy:  2448 ,  0.0002755937424009078
Landsat_8_0_20190108.npy:  179982 ,  0.020262219340196157
Landsat_8_0_20190201.npy:  1132086 ,  0.12744927183810217
Landsat_8_0_20190225.npy:  0 ,  0.0
Landsat_8_0_20190321.npy:  2953 ,  0.00033244620968540887
Landsat_8_0_20190414.npy:  696 ,  7.835508362378752e-05
Landsat_8_0_20190508.npy:  475728 ,  0.05355705060657642
Landsat_8_0_20190601.npy

#### Convert to .npy files

In [12]:
for path in tqdm(cdl_paths):
  new_img = gdal.Open(path)
  band_cnt = new_img.RasterCount
  channel_list = []
  is_nan_too_much = False
  for b in range(1, band_cnt + 1):
    img_array = np.array(new_img.GetRasterBand(b).ReadAsArray())
    nan_percent = np.count_nonzero(np.isnan(img_array)) / (img_array.shape[0]*img_array.shape[1])
    # print('before: ',nan_percent)
    if nan_percent > 0.3:
      is_nan_too_much = True
      NAN_image_path.append(path)
    img_array = replace_nan(img_array)
    nan_percent = np.count_nonzero(np.isnan(img_array)) / (img_array.shape[0]*img_array.shape[1])
    # print('after: ',nan_percent)
    channel_list.append(img_array)
  output = np.moveaxis(np.stack(channel_list, axis=0), 0, -1).squeeze()
  output_path = path.replace('.tif', '.npy')
  # print(output_path)
  # print(output.shape)
  np.save(output_path, output)

100%|██████████| 64/64 [02:49<00:00,  2.64s/it]


## Make the dataset

In [ ]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
years = [2017, 2018, 2019, 2020]
site_range = [1, 2, 3, 4, 5, 6]
All_features_path = os.path.join(base_path, "All_features")
SAR_features_path = os.path.join(base_path, "SAR_features")
Sp_features_path = os.path.join(base_path, "Sp_features")
CDL_path = os.path.join(base_path, "CDL_label")
# for satellite in satellites:
#   cur_path = None
#   if satellite == 'Landsat_8':
#     cur_path = L8_path
#   elif satellite == 'S1':
#     cur_path = S1_path
for site in site_range:
  # print(cur_path)
  cur_path = None
  site_path = str(site)
  # print(cur_path)
  for year in years:
    year_path = os.path.join(site_path,str(year))
    cdlpath = "CDL_" + str(site) + '_' + str(year) + '.npy'
    cdlpath = os.path.join(year_path, cdlpath)
    cdlpath = os.path.join(L8_path, cdlpath)
    label = np.load(cdlpath)
    start_date = date(year, 9, 10)
    SAR_list = []
    SP_list = []
    for advancement in range(1, 11):
      cur_time = start_date + datetime.timedelta(days=advancement*24)
      cur_time_str = cur_time.strftime("%Y%m%d")
      S1_cur_image_path = os.path.join(year_path, 'S1_' + str(site) + '_'+ cur_time_str + '.npy')
      L8_cur_image_path = os.path.join(year_path, 'Landsat_8_' + str(site) + '_'+ cur_time_str + '.npy')
      S1_img_path = os.path.join(S1_path, S1_cur_image_path)
      L8_img_path = os.path.join(L8_path, L8_cur_image_path)
      SP_list.append(np.load(L8_img_path))
      SAR_list.append(np.load(S1_img_path))
    SAR_img = np.stack(SAR_list, axis=-1)
    SP_img = np.stack(SP_list, axis=-1)
    All_img = np.concatenate((SP_img, SAR_img), axis=2)
    img_cnt = 0
    x, y, c, t = All_img.shape
    x_steps = x//IMG_WIDTH
    y_steps = y//IMG_HEIGHT
    for i in range(x_steps):
      for j in range(y_steps):
        break_flg = 0
        for w in range(i*IMG_WIDTH, i*IMG_WIDTH + IMG_WIDTH//2):
          if break_flg or w > x - 1:
            break
          for h in range(j*IMG_HEIGHT, j*IMG_HEIGHT + IMG_HEIGHT//2):
            if h > y-1:
              break
            if not np.any(np.isnan(All_img[w : w+IMG_WIDTH, h : h+IMG_HEIGHT, :, :])) and not np.any(np.isnan(label[w : w+IMG_WIDTH, h : h+IMG_HEIGHT,])):
              all_fea_path = os.path.join(All_features_path, str(site) + '_'+ str(year) +"_" +str(img_cnt) + '.npy')
              label_path = os.path.join(CDL_path, str(site) + '_'+ str(year) +"_"+ str(img_cnt) + '.npy')
              SAR_img_path = os.path.join(SAR_features_path, str(site) + '_'+ str(year) +"_"+ str(img_cnt) + '.npy')
              Sp_img_path = os.path.join(Sp_features_path, str(site) + '_'+ str(year) +"_"+ str(img_cnt) + '.npy')
              np.save(all_fea_path, All_img[w : w+IMG_WIDTH, h : h+IMG_HEIGHT, :, :])
              np.save(SAR_img_path, SAR_img[w : w+IMG_WIDTH, h : h+IMG_HEIGHT, :, :])
              np.save(Sp_img_path, SP_img[w : w+IMG_WIDTH, h : h+IMG_HEIGHT, :, :])
              np.save(label_path, label[w : w+IMG_WIDTH, h : h+IMG_HEIGHT])
              break_flg = True
              img_cnt += 1  
              file_path = os.path.join(CDL_path, str(site) + '_'+ str(year) + '.txt')
              f = open(file_path, "a")
              f.write(str(img_cnt)+": [ "+str(w)+", "+str(h) + " ]   [ " + str(w + IMG_WIDTH -1) + ", " + str(h + IMG_HEIGHT -1) + " ]\n")
              f.close()
              break
    print(str(site)," ",  str(year), ": ", str(img_cnt), "images, ", img_cnt/(x_steps*y_steps))


1   2017 :  49 images,  1.0
1   2018 :  47 images,  0.9591836734693877
1   2019 :  49 images,  1.0
1   2020 :  49 images,  1.0
2   2017 :  54 images,  0.9642857142857143
2   2018 :  54 images,  0.9642857142857143
2   2019 :  54 images,  0.9642857142857143
2   2020 :  54 images,  0.9642857142857143
3   2017 :  49 images,  1.0
3   2018 :  49 images,  1.0
3   2019 :  49 images,  1.0
3   2020 :  49 images,  1.0
4   2017 :  46 images,  0.8214285714285714
4   2018 :  46 images,  0.8214285714285714
4   2019 :  46 images,  0.8214285714285714
4   2020 :  46 images,  0.8214285714285714
5   2017 :  67 images,  0.8271604938271605
5   2018 :  67 images,  0.8271604938271605
5   2019 :  67 images,  0.8271604938271605
5   2020 :  67 images,  0.8271604938271605
6   2017 :  49 images,  1.0
6   2018 :  49 images,  1.0
6   2019 :  49 images,  1.0
6   2020 :  49 images,  1.0
